In [1]:
import requests
import datetime
from urllib.parse import urlencode

In [2]:
import base64

In [3]:
client_id = 'e9a3d54e253d4efc91ada2f4cdd67cfc'
client_secret = '74d8228a44694380a45feafaa516ac7d'

In [4]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("Se debe setear el client_id y client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
        
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("No se pudo autenticar.")
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now or token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def search(self, query=None, search_type='artist'):
        if query == None:
            raise Exception("Se necesita un parametro de busqueda")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        query_params = urlencode({"q": query, "type": search_type.lower()})
        return self.base_search(query_params)

In [5]:
client = SpotifyAPI(client_id, client_secret)

In [7]:
client.search(query="Someone Else", search_type="Track")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Someone+Else&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4aKdmOXdUKX07HVd3sGgzw'},
       'href': 'https://api.spotify.com/v1/artists/4aKdmOXdUKX07HVd3sGgzw',
       'id': '4aKdmOXdUKX07HVd3sGgzw',
       'name': 'Rezz',
       'type': 'artist',
       'uri': 'spotify:artist:4aKdmOXdUKX07HVd3sGgzw'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/4fv1OFJywZ7DHCz3mVQQ45'},
       'href': 'https://api.spotify.com/v1/artists/4fv1OFJywZ7DHCz3mVQQ45',
       'id': '4fv1OFJywZ7DHCz3mVQQ45',
       'name': 'Grabbitz',
       'type': 'artist',
       'uri': 'spotify:artist:4fv1OFJywZ7DHCz3mVQQ45'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
   

In [ ]:
client.get_artist('3TVXtAsR1Inumwj472S9r4')